In [1]:
import pandas as pd
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
sys.argv[1] = 'ratings.jsonl'
sys.argv[2] = 'content.jsonl'
sys.argv[3] = 'targets.csv'

Ratings = pd.read_json(sys.argv[1], lines=True)
Content = pd.read_json(sys.argv[2], lines=True)
Target = pd.read_csv(sys.argv[3])

submission = pd.DataFrame(columns=['UserId','ItemId'])


In [3]:
Users = Target["UserId"].copy()
Users = Users.drop_duplicates()
Users = Users.reset_index(drop=True)

Items = Content["ItemId"].copy()
Items = Items.drop_duplicates()
Items = Items.reset_index(drop=True)



In [6]:
Content

,ItemId,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons,Season,Episode,seriesID
0,c9f0f895fb,Edison Kinetoscopic Record of a Sneeze,1894,N/A,09 Jan 1894,1 min,"Documentary, Short",William K.L. Dickson,N/A,Fred Ott,...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
1,d3d9446802,Leaving the Factory,1895,Not Rated,22 Mar 1895,1 min,"Documentary, Short",Louis Lumière,N/A,N/A,...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
2,c20ad4d76f,The Arrival of a Train,1896,Not Rated,25 Jan 1896,1 min,"Documentary, Short","Auguste Lumière, Louis Lumière",N/A,"Madeleine Koehler, Marcel Koehler, Mrs. August...",...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
3,8e296a067a,The Oxford and Cambridge University Boat Race,1895,N/A,N/A,N/A,"Short, News, Sport",Birt Acres,N/A,N/A,...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
4,54229abfcf,The House of the Devil,1896,Not Rated,24 Dec 1896,3 min,"Short, Horror",Georges Méliès,Georges Méliès,"Jehanne d'Alcy, Jules-Eugène Legris, Georges M...",...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38007,6c0ffc79d0,Yara,2021,TV-14,05 Nov 2021,96 min,"Crime, Drama, Thriller",Marco Tullio Giordana,"Graziano Diana, Giacomo Martelli","Isabella Ragonese, Alessio Boni, Thomas Trabacchi",...,movie,05 Nov 2021,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
38008,e02f371f8c,Lords of Scam,2021,N/A,03 Nov 2021,105 min,"Documentary, Crime",Guillaume Nicloux,Olivier Bouchara,N/A,...,movie,03 Nov 2021,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
38009,8c2a2a22b8,Cash,2021,N/A,19 Nov 2021,118 min,"Comedy, Drama",Rishab Seth,"Vishesh Bhatt, Rishab Seth, Aarsh Vora","Amol Parashar, Smiriti Kalra, Gulshan Grover",...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
38010,ae74ba6bb7,Sompoy,2021,N/A,04 Nov 2021,120 min,"Comedy, Romance","Anawat Phromchae, Aroonakorn Pick","Anawat Phromchae, Aroonakorn Pick","Pijakkana Wongsarattanasin, Tanapol Jarujittra...",...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN


In [7]:
tf= TfidfVectorizer(stop_words='english')
matrix = tf.fit_transform(Content['Genre'])

<38012x31 sparse matrix of type '<class 'numpy.float64'>'
	with 86625 stored elements in Compressed Sparse Row format>

In [8]:
cosine_sim = linear_kernel(matrix,matrix)

In [14]:
def get_rec():
    
    sim_scores = list(enumerate(cosine_sim[movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


0.7257595942179731

In [9]:
indices = pd.Series(Users.index, index=Users)
indices

UserId
0006246bee       0
001fca6b30       1
0038c2d256       2
003a8eb481       3
003bed21fd       4
              ... 
ffd124c228    6157
ffd1ca3f09    6158
ffdc324c91    6159
ffdd1df879    6160
fffffe98d0    6161
Length: 6162, dtype: int64

In [10]:
submission.to_csv('submission.csv',index=False)